# Video Analyzer with Code

In [1]:
!pip install -U -q google-genai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.7/118.7 kB 2.0 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

In [ ]:
from typing import List
from google import genai
from google.genai import types

client = genai.Client(api_key=GOOGLE_API_KEY)

In [ ]:
from IPython.display import Markdown

In [ ]:
MODEL_ID = "gemini-2.0-flash-exp"

## Upload Our Video

In [ ]:
import pathlib

# Use Path instead of path
img_path = pathlib.Path('/content/drive/MyDrive/videoplayback.m4a')

In [ ]:
#Upload the file using the API
file_upload = client.files.upload(path=img_path)

In [ ]:
file_upload

File(name='files/t43xlkldxvpy', display_name=None, mime_type='audio/mp4', size_bytes=7585312, create_time=datetime.datetime(2025, 1, 8, 11, 54, 31, 165191, tzinfo=TzInfo(UTC)), expiration_time=datetime.datetime(2025, 1, 10, 11, 54, 31, 136698, tzinfo=TzInfo(UTC)), update_time=datetime.datetime(2025, 1, 8, 11, 54, 31, 165191, tzinfo=TzInfo(UTC)), sha256_hash=b'77c5df1135c1c7a45f29b5b858751d80f78513276ca0d2e79aeeb3554e4bff12', uri='https://generativelanguage.googleapis.com/v1beta/files/t43xlkldxvpy', state='ACTIVE', video_metadata=None, error=None)

In [ ]:
file_upload.state

'ACTIVE'

In [ ]:
import time
#prepare the file to be uploded
while file_upload.state == "PROCESSING":
  print("Waiting for video to be processed.")
  time.sleep(10)
  file_upload = client.files.get(name=file_upload.name)

if file_upload.state == "FAILED":
  raise ValueError(file_upload.state)
  print(f'video processing complete:' + file_upload.uri)

In [ ]:
print(file_upload.state)

ACTIVE


## Basic Calling a prompt on the video

In [ ]:
SYSTEM_PROMPT = "When given a video and a query, call the relevant function only once with the appropriate timecodes and text for the video "

In [ ]:
USER_PROPMT = "For each scene in this video, generate captions that describe the scene along with any spoken text placed in quotation marks. Place each caption into an object sent to set_timecodes with the timecode of the caption in the video. "

In [ ]:
USER_PROPMT = "For each scene in this video, generate captions that describe the scene along with any spoken text placed in quotation marks. Place each caption into an object with the timecode of the caption in the video"

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        types.Content(
           role="user",
            parts=[
                types.Part.from_uri(
                    file_uri=file_upload.uri,
                    mime_type=file_upload.mime_type),
                ]),
        USER_PROPMT,
    ],
   config=types.GenerateContentConfig(
       system_instruction=SYSTEM_PROMPT,
       temperature=0.0,
   ),
)

Markdown(response.text)

```json
[
  {
    "timecode": "0:00",
    "text": "A person is speaking in front of a screen showing code. \"When we talked with Replit about how they built their agent, one of the things that they highlighted was the importance of human in the loop.\""
  },
  {
    "timecode": "0:10",
    "text": "The screen shows a conversation between a user and an agent. \"And so you can see here on the side that they're showing a conversation of what's happening, but they're also letting the user approve the plan.\""
  },
  {
    "timecode": "0:19",
    "text": "The speaker continues to talk about human in the loop interactions. \"And so this human in the loop interaction pattern is something that we've seen become really important and really common when building complex agent applications.\""
  },
  {
    "timecode": "0:29",
    "text": "The speaker talks about LangGraph. \"And we've made LangGraph really, really good for these human in the loop interactions. And today we're adding a new tool called interrupt to make it even easier.\""
  },
  {
    "timecode": "0:39",
    "text": "The speaker talks about the persistence layer of LangGraph. \"So let's take a look at that. To maybe set the stage a bit, the reason that LangGraph is so well built for human in the loop interaction patterns is the persistence layer that it's built on.\""
  },
  {
    "timecode": "0:49",
    "text": "The speaker explains how the persistence layer works. \"So when you use LangGraph, if you pass in a checkpointer, then what happens is that between each step, the state of the graph is read from the checkpoint and then written to the checkpoint when it ends.\""
  },
  {
    "timecode": "1:01",
    "text": "The speaker explains how the persistence layer allows for stopping and resuming execution. \"This means that you can stop execution anywhere and you can then resume an arbitrary time length later because you have that checkpoint. You have that exact state of the graph at that period of time.\""
  },
  {
    "timecode": "1:11",
    "text": "The speaker explains how the persistence layer allows for editing. \"It also means that the human can edit that checkpoint, update it, add some different values, and then restart the graph from that new updated edited checkpoint.\""
  },
  {
    "timecode": "1:22",
    "text": "The speaker describes the persistence layer as a scratchpad. \"You can think of it almost as a scratchpad for human agent collaboration.\""
  },
  {
    "timecode": "1:27",
    "text": "The speaker explains that the persistence layer is a core part of LangGraph. \"So for everything that I'm going to talk about, this is enabled by the persistence layer that's a core part of LangGraph and has been from the beginning.\""
  },
  {
    "timecode": "1:34",
    "text": "The speaker introduces the interrupt function. \"The new functionality we're adding is this interrupt function.\""
  },
  {
    "timecode": "1:39",
    "text": "The speaker explains how the interrupt function is similar to the input function in Python. \"This function's kind of similar to the input function in Python.\""
  },
  {
    "timecode": "1:45",
    "text": "The speaker explains how to use the interrupt function. \"So what it means is that inside a node, you can use interrupt and you can pass in any data into interrupt.\""
  },
  {
    "timecode": "1:52",
    "text": "The speaker explains how the interrupt function saves data. \"That'll be saved as part of the persistence layer and the node will be interrupted right there.\""
  },
  {
    "timecode": "1:58",
    "text": "The speaker explains how to resume the graph. \"You can then reinvoke the graph, and I'll show how to do that in a bit. And when you get back the result, that'll be written as this value. This is very similar to how input works in Python.\""
  },
  {
    "timecode": "2:09",
    "text": "The speaker explains the difference between input and interrupt. \"The main difference is that while input is really useful for when you're running a graph in memory, in the CLI, in a notebook, it doesn't really work in production at all. Interrupt does.\""
  },
  {
    "timecode": "2:20",
    "text": "The speaker explains how to use interrupt in a graph. \"So once you have this node where you're using interrupt, you can then build your graph. Notice that you will need to pass in a checkpointer because this relies on that persistence layer. You can invoke it the first time, pass in anything, and then it will eventually interrupt it there.\""
  },
  {
    "timecode": "2:36",
    "text": "The speaker explains how to resume the graph with a command. \"And then you can resume by passing in this command thing with the value that you want to be written to this value. So, let's take a look at this in a little bit more detail and see it in action.\""
  },
  {
    "timecode": "2:47",
    "text": "The speaker introduces a simple graph example. \"Let's take a look at a really simple graph here that shows off how to use interrupt.\""
  },
  {
    "timecode": "2:52",
    "text": "The speaker explains the code for the simple graph. \"So I'm using the state graph, I'm using messages state. I only have one node. It's this human node. What I'm going to do inside this node is I'm going to use interrupt.\""
  },
  {
    "timecode": "3:01",
    "text": "The speaker explains how the interrupt function is used in the node. \"Into interrupt, I'm going to pass this string. The string is going to be formatted with the messages that are in the state up to that point in time. I'm going to get back this value from interrupt and I'm going to update the state based on that value. So I'm going to make this an AI message.\""
  },
  {
    "timecode": "3:16",
    "text": "The speaker explains how the graph is built. \"I'm going to build this graph, I'm going to use my checkpointer, it's an in memory checkpointer. I'm going to create this graph, I'm going to add a node, it's this only node, I'm going to set it as the entry point, going to build it, going to create this thread config, and then I'm going to invoke it with the initial messages. So role user, content hi.\""
  },
  {
    "timecode": "3:33",
    "text": "The speaker explains how to get the state of the graph. \"Great. Let's now get the state of the graph at this point in time.\""
  },
  {
    "timecode": "3:38",
    "text": "The speaker shows the state of the graph. \"We can see that so far, the state values are just a single message. It's just a human message, it's what I passed in.\""
  },
  {
    "timecode": "3:46",
    "text": "The speaker shows the tasks on the state. \"We can see that there are a few tasks on the state. Well, there's one task on the state. And this task has some interrupts, one interrupt. And this interrupt is the value that I passed in earlier. What should I say in response to? And then this is this is the state that I that I formatted into the string.\""
  },
  {
    "timecode": "4:02",
    "text": "The speaker explains how the interrupt is used. \"So this shows that this thread at this point in time is interrupted. And in that interrupt, I have information about why it's interrupted. And so you can put anything in this interrupt value that helps communicate to the end user why it is interrupted and how they should respond.\""
  },
  {
    "timecode": "4:18",
    "text": "The speaker explains how to respond to the interrupt. \"And here is what it looks like to respond. So, I'm going to respond with the command tool, and we introduced this a few days ago. I'm going to pass in resume, and then how how is it going? This is going to be the value that will get saved right there.\""
  },
  {
    "timecode": "4:33",
    "text": "The speaker shows the result of the resume command. \"So I'm going to pass that in, and then the graph is going to finish, and I can see that it added here this AI message with the content how's it going? This is what I passed in. This is what gets saved to value, and in turn, this is what updated the state.\""
  },
  {
    "timecode": "4:46",
    "text": "The speaker summarizes the example. \"So this is a really simple example of how interrupt works in practice.\""
  },
  {
    "timecode": "4:50",
    "text": "The speaker introduces common use cases for interrupt. \"Let's now talk about some common use cases for this.\""
  },
  {
    "timecode": "4:54",
    "text": "The speaker explains the first use case, approve or reject. \"So first is kind of like an approve or reject step. The LLM makes some decision, this can either just be a response or it can be a function call, and the human approves it. If it approves it, then it goes to some node, or it rejects it. If it rejects it, it goes to a different node. This is a pretty common pattern.\""
  },
  {
    "timecode": "5:11",
    "text": "The speaker explains the second use case, reviewing and editing the state. \"Another one is reviewing and editing the state. This is a little bit more involved. So now you have the state of the graph, and so it could be the response of an LLM or it could be documents or it could be anything. And the human's going to go in, they're going to review that whole state, and then they're going to pass in some information that's going to update that state optionally, and then after that's done, it will go to the next node.\""
  },
  {
    "timecode": "5:31",
    "text": "The speaker explains how this use case is useful. \"So this is useful for correcting mistakes.\""
  },
  {
    "timecode": "5:34",
    "text": "The speaker explains how this use case is useful. \"So this can be useful for correcting mistakes or updating the state with additional information.\""
  },
  {
    "timecode": "5:40",
    "text": "The speaker explains the third use case, reviewing tool calls. \"This is kind of similar to the previous use case of approve or reject, but just calling it out that reviewing tool calls is one of the main use cases for human in the loop that we see.\""
  },
  {
    "timecode": "5:49",
    "text": "The speaker explains how reviewing tool calls is useful. \"So the LLM may generate a tool call or multiple tool calls with multiple arguments, and having the human in the loop to decide whether that tool call is correct and whether it should be executed can be really, really important, particularly in critical applications where that tool call is requested by the LLM may be sensitive or require human oversight.\""
  },
  {
    "timecode": "6:10",
    "text": "The speaker explains the fourth use case, multi-turn conversations. \"Finally, this can be a useful design pattern for when you're building multi-turn conversations in multi-agent setups.\""
  },
  {
    "timecode": "6:17",
    "text": "The speaker explains how to use the persistence layer for single agent conversations. \"So if you're doing a multi-turn conversation with a single agent, then you can just use the persistence layer as normal and use the memory that way.\""
  },
  {
    "timecode": "6:24",
    "text": "The speaker explains how to use interrupt for multi-agent conversations. \"When you're doing a multi-turn conversation with multiple agents, then it's often nice to have a human node and just interrupt there rather than finishing the graph run, because then after that human node, you know exactly where you are in any of the three or four, however many agents you have, and you can return to that agent after you respond rather than having to do some routing at the beginning.\""
  },
  {
    "timecode": "6:44",
    "text": "The speaker explains that the human in the loop can be done by the end user or a third party. \"Note that for all of these use cases, the human in the loop interaction pattern can be done either by the end user or by a third party who's kind of like reviewing and monitoring the system. So there's a lot of optionality here.\""
  },
  {
    "timecode": "6:58",
    "text": "The speaker summarizes the importance of human in the loop. \"We think human in the loop design patterns are crucial for building reliable and useful agents. We want LangGraph to be the best place to build these types of human in the loop agents. So please check this out and let us know if you have any feedback. Thanks for watching.\""
  }
]
```

## Add in the Function Calls to get back the data in a way we expect it

In [ ]:
set_timecodes = types.FunctionDeclaration(
    name="set_timecodes",
    description="Set the timecodes for the video associated text",
    parameters={
        "type": "OBJECT",
        "properties": {
            "timecodes": {
                "type": "ARRAY",
                "items": {
                    "type" : "OBJECT",
                    "properties" : {
                        "time": {"type": "STRING"},
                        "text": {"type": "STRING" },
                    },
                    "required": ["time", "text"],
                }
            }
        },
        "required": ["timecodes"]
    }
)

set_timecodes_with_objects = types.FunctionDeclaration(
    name="set_timecodes_with_objects",
    description="Set the timecodes for the video with associated text and object list",
    parameters={
        "type": "OBJECT",
        "properties": {
            "timecodes": {
                "type": "ARRAY",
                "items": {
                    "type" : "OBJECT",
                    "properties" : {
                        "time": {"type": "STRING"},
                        "text": {"type": "STRING" },
                        "object":{
                            "type": "ARRAY",
                            "items":{"type":"STRING"},
                        },
                        },
                       "required": ["time", "text", "object"],
           }
        }
    },
        "required": ["timecodes"],
    }
)




set_timecodes_with_numeric_values =  types.FunctionDeclaration(
    name="set_timecodes_with_numeric_values",
    description="Set the timecodes for the video with associated numeric values",
    parameters={
        "type": "OBJECT",
        "properties": {
            "timecodes": {
                "type": "ARRAY",
                "items": {
                    "type" : "OBJECT",
                    "properties" : {
                        "time": {"type": "STRING"},
                        "value": {"type": "NUMBER" },
                        },
                       "required": ["time", "value"],
                    }
        }
    },
        "required": ["timecodes"],
    }
)


set_timecodes_with_description = types.FunctionDeclaration(
    name="set_timecodes_with_descriptions",
    description="Set the timecodes for the video with associated spoken text and visual descriptions",
    parameters={
        "type": "OBJECT",
        "properties": {
            "timecodes": {
                "type": "ARRAY",
                "items": {
                    "type" : "OBJECT",
                    "properties" : {
                        "time": {"type": "STRING"},
                        "spoken_text": {"type": "STRING" },
                        "visual_description": {"type": "STRING"},
                    },
                    "required": ["time", "spoken_text", "visual_description"],
                }
            }
        },
        "required": ["timecodes"]
    }
)

In [ ]:
video_tools = types.Tool(
    function_declarations=[set_timecodes, set_timecodes_with_objects, set_timecodes_with_numeric_values],
)

In [ ]:
def set_timecodes_func(timecodes):
  return [{**t, "text": t["text"].replace("\\","'")} for t in timecodes]
def set_timecodes_with_objects_func(timecodes):
  return [{**t, "text": t["text"].replace("\\","'")} for t in timecodes]
def set_timecodes_with_description_func(timecodes):
  return [{**t, "text": t["spoken_text"].replace("\\","'")} for t in timecodes]

In [ ]:
USER_PROPMT

'For each scene in this video, generate captions that describe the scene along with any spoken text placed in quotation marks. Place each caption into an object with the timecode of the caption in the video'

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        types.Content(
           role="user",
            parts=[
                types.Part.from_uri(
                    file_uri=file_upload.uri,
                    mime_type=file_upload.mime_type),
                ]),
        USER_PROPMT,
    ],
   config=types.GenerateContentConfig(
       system_instruction=SYSTEM_PROMPT,
       tools=[video_tools],
       temperature=0,
   ),
)

response.candidates[0].content.parts[0].function_call

FunctionCall(id=None, args={'timecodes': [{'text': 'The video begins with a person speaking about Replit and their agent.', 'time': '0:00'}, {'text': 'The speaker highlights the importance of human in the loop and shows a conversation where the user approves the plan.', 'time': '0:06'}, {'text': 'The speaker says that human in the loop interaction is important when building complex agent applications.', 'time': '0:18'}, {'text': 'The speaker introduces a new tool called interrupt to make human in the loop interactions easier.', 'time': '0:27'}, {'text': 'The speaker explains that LangGraph is well built for human in the loop interactions because of the persistence layer.', 'time': '0:34'}, {'time': '0:45', 'text': 'The speaker explains that when using LangGraph, the state of the graph is read from the checkpoint and written to the checkpoint when it ends.'}, {'time': '0:53', 'text': 'The speaker explains that you can stop execution anywhere and resume later because of the checkpoint.'}

In [ ]:
Markdown(response.candidates[0].content.parts[0].text)

<IPython.core.display.Markdown object>

In [ ]:
USER_PROPMT = "For each scene in this video, generate captions that describe the scene along with any spoken text placed in quotation marks. place each caption into an object sent to set_timecodes with the timecode of the caption in the video. "

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        types.Content(
           role="user",
            parts=[
                types.Part.from_uri(
                    file_uri=file_upload.uri,
                    mime_type=file_upload.mime_type),
                ]),
        USER_PROPMT,
    ],
   config=types.GenerateContentConfig(
       system_instruction=SYSTEM_PROMPT,
       tools=[video_tools],
       temperature=0,
   ),
)

response.candidates[0].content.parts[0].function_call

FunctionCall(id=None, args={'timecodes': [{'time': '0:00', 'text': 'The video begins with a person speaking about Replit and their agent.'}, {'text': 'The speaker highlights the importance of human in the loop and shows a conversation where the user approves the plan.', 'time': '0:06'}, {'text': 'The speaker mentions that human in the loop interaction is important when building complex agent applications.', 'time': '0:18'}, {'text': "The speaker introduces a new tool called 'interrupt' to make human in the loop interactions easier.", 'time': '0:27'}, {'text': 'The speaker explains that LangGraph is well-built for human in the loop interactions due to its persistence layer.', 'time': '0:33'}, {'time': '0:45', 'text': 'The speaker explains that when using LangGraph, the state of the graph is read from and written to a checkpoint between each step.'}, {'time': '0:58', 'text': 'The speaker explains that the human can edit the checkpoint, update it, and restart the graph from the updated ch

In [ ]:
response.candidates[0].content.parts[0].function_call.name

'set_timecodes'

In [ ]:
set_timecodes_func(response.candidates[0].content.parts[0].function_call.args['timecodes'])

[{'time': '0:00',
  'text': 'The video begins with a person speaking about Replit and their agent.'},
 {'text': 'The speaker highlights the importance of human in the loop and shows a conversation where the user approves the plan.',
  'time': '0:06'},
 {'text': 'The speaker mentions that human in the loop interaction is important when building complex agent applications.',
  'time': '0:18'},
 {'text': "The speaker introduces a new tool called 'interrupt' to make human in the loop interactions easier.",
  'time': '0:27'},
 {'text': 'The speaker explains that LangGraph is well-built for human in the loop interactions due to its persistence layer.',
  'time': '0:33'},
 {'time': '0:45',
  'text': 'The speaker explains that when using LangGraph, the state of the graph is read from and written to a checkpoint between each step.'},
 {'time': '0:58',
  'text': 'The speaker explains that the human can edit the checkpoint, update it, and restart the graph from the updated checkpoint.'},
 {'text'